In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
torch.manual_seed(222)
num_epoch = 5000
Learning_Rate = 0.01

In [2]:
training_data = pd.read_csv("G:/내 드라이브/데이콘/와인 품질 경진대회/data/Training.csv")
test_data = pd.read_csv("G:/내 드라이브/데이콘/와인 품질 경진대회/data/Test.csv")

In [3]:
training_data.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,1436,6,7.1,0.18,0.26,1.3,0.041,20.0,71.0,0.99260,3.04,0.74,9.9,white
1,1076,6,6.2,0.27,0.43,7.8,0.056,48.0,244.0,0.99560,3.10,0.51,9.0,white
2,1961,5,7.7,0.25,0.30,7.8,0.038,67.0,196.0,0.99555,3.10,0.50,10.1,white
3,4709,5,9.4,0.34,0.37,2.2,0.075,5.0,13.0,0.99800,3.22,0.62,9.2,red
4,330,6,7.4,0.30,0.49,8.2,0.055,49.0,188.0,0.99740,3.52,0.58,9.7,white


In [4]:
List_of_index = training_data.columns
print(List_of_index)

Index(['index', 'quality', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'type'],
      dtype='object')


In [5]:
y = training_data['type']

# white = 0, red = 1로 변환
white = y == "white"
y[white] = 0.
red = y == "red"
y[red] = 1.

C:\Users\USER\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\USER\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
x_ar = np.array(training_data.drop(["type"], axis=1)).astype(float)
y_ar = np.array(y).astype(float)
print(x_ar[:,1].shape)
print(y_ar.shape)
correlation = np.zeros((x_ar.shape[1],2))
for i in range(x_ar.shape[1]):
    print(np.corrcoef(x_ar[:,i],y_ar))
    correlation[i,0]=i
    correlation[i,1]=np.corrcoef(x_ar[:,i],y_ar)[0,1]

(3847,)
(3847,)
[[ 1.         -0.02186959]
 [-0.02186959  1.        ]]
[[ 1.         -0.11609489]
 [-0.11609489  1.        ]]
[[1.         0.48673586]
 [0.48673586 1.        ]]
[[1.         0.65460654]
 [0.65460654 1.        ]]
[[ 1.         -0.20524908]
 [-0.20524908  1.        ]]
[[ 1.         -0.34774176]
 [-0.34774176  1.        ]]
[[1.         0.50378365]
 [0.50378365 1.        ]]
[[ 1.         -0.45991678]
 [-0.45991678  1.        ]]
[[ 1.         -0.69963307]
 [-0.69963307  1.        ]]
[[1.         0.38216916]
 [0.38216916 1.        ]]
[[1.         0.32027805]
 [0.32027805 1.        ]]
[[1.         0.48112862]
 [0.48112862 1.        ]]
[[ 1.         -0.02890291]
 [-0.02890291  1.        ]]


In [7]:
# -1에 가까우면 반비례
# 1에 가까우면 비례
# 0에 가까우면 상관 없음
# -> 절대값 높은 것?
print(correlation)
print(abs(correlation))
correlation_sort = np.sort(abs(correlation[:,1]))
print(correlation_sort)
first_corr = List_of_index[np.where(abs(correlation[:,1])==correlation_sort[-1])]
second_corr = List_of_index[np.where(abs(correlation[:,1])==correlation_sort[-2])]
print("가장 높은 상관계수의 index : ",List_of_index[np.where(abs(correlation[:,1])==correlation_sort[-1])])
print("두번째로 높은 상관계수의 index : ",List_of_index[np.where(abs(correlation[:,1])==correlation_sort[-2])])

[[ 0.         -0.02186959]
 [ 1.         -0.11609489]
 [ 2.          0.48673586]
 [ 3.          0.65460654]
 [ 4.         -0.20524908]
 [ 5.         -0.34774176]
 [ 6.          0.50378365]
 [ 7.         -0.45991678]
 [ 8.         -0.69963307]
 [ 9.          0.38216916]
 [10.          0.32027805]
 [11.          0.48112862]
 [12.         -0.02890291]]
[[ 0.          0.02186959]
 [ 1.          0.11609489]
 [ 2.          0.48673586]
 [ 3.          0.65460654]
 [ 4.          0.20524908]
 [ 5.          0.34774176]
 [ 6.          0.50378365]
 [ 7.          0.45991678]
 [ 8.          0.69963307]
 [ 9.          0.38216916]
 [10.          0.32027805]
 [11.          0.48112862]
 [12.          0.02890291]]
[0.02186959 0.02890291 0.11609489 0.20524908 0.32027805 0.34774176
 0.38216916 0.45991678 0.48112862 0.48673586 0.50378365 0.65460654
 0.69963307]
가장 높은 상관계수의 index :  Index(['total sulfur dioxide'], dtype='object')
두번째로 높은 상관계수의 index :  Index(['volatile acidity'], dtype='object')


In [8]:
x = training_data[[first_corr[0],second_corr[0]]]
TEST_DATA = test_data[[first_corr[0],second_corr[0]]]

In [9]:


X = torch.tensor(x.values,dtype=torch.float32)
Y = torch.reshape(torch.tensor(y,dtype=torch.float32),[y.shape[0],1])
TEST = torch.tensor(TEST_DATA.values,dtype=torch.float32)

print(X.shape)
print(Y.shape)
print(TEST.shape)

torch.Size([3847, 2])
torch.Size([3847, 1])
torch.Size([1650, 2])


In [10]:
# Binary Crossentropy 손실 = loss(x,y) = BCE
model = nn.Sequential(nn.Linear(X.shape[1], Y.shape[1]), nn.Sigmoid())
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters() ,lr = Learning_Rate)

Loss = np.zeros((num_epoch,1))
Accuracy = np.zeros((num_epoch,1))

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_func(output,Y)
    loss.backward()
    optimizer.step()
    # Accuracy Calculate
    temp1 = torch.le(output,0.5)
    output[temp1] = 0
    temp2 = torch.gt(output,0.5)
    output[temp2] = 1
    accuracy = torch.divide(torch.count_nonzero(torch.eq(output,Y)),Y.shape[0]*Y.shape[1])
    
    Loss[i,0] = loss.data
    Accuracy[i,0] = accuracy
    if i % 10 == 0:
        print(i,"Loss : ",loss.data,"Accuracy : ",accuracy.data)
        

0 Loss :  tensor(72.6321) Accuracy :  tensor(0.2464)
10 Loss :  tensor(46.6333) Accuracy :  tensor(0.2464)
20 Loss :  tensor(2.8880) Accuracy :  tensor(0.2095)
30 Loss :  tensor(0.8486) Accuracy :  tensor(0.7536)
40 Loss :  tensor(1.0931) Accuracy :  tensor(0.7536)
50 Loss :  tensor(0.9846) Accuracy :  tensor(0.7536)
60 Loss :  tensor(0.7302) Accuracy :  tensor(0.7536)
70 Loss :  tensor(0.4558) Accuracy :  tensor(0.7536)
80 Loss :  tensor(0.4333) Accuracy :  tensor(0.7536)
90 Loss :  tensor(0.3870) Accuracy :  tensor(0.7536)
100 Loss :  tensor(0.3835) Accuracy :  tensor(0.7536)
110 Loss :  tensor(0.3712) Accuracy :  tensor(0.7541)
120 Loss :  tensor(0.3630) Accuracy :  tensor(0.7572)
130 Loss :  tensor(0.3557) Accuracy :  tensor(0.7668)
140 Loss :  tensor(0.3481) Accuracy :  tensor(0.7829)
150 Loss :  tensor(0.3410) Accuracy :  tensor(0.7988)
160 Loss :  tensor(0.3341) Accuracy :  tensor(0.8092)
170 Loss :  tensor(0.3274) Accuracy :  tensor(0.8196)
180 Loss :  tensor(0.3209) Accuracy :

1520 Loss :  tensor(0.1565) Accuracy :  tensor(0.9465)
1530 Loss :  tensor(0.1564) Accuracy :  tensor(0.9465)
1540 Loss :  tensor(0.1562) Accuracy :  tensor(0.9467)
1550 Loss :  tensor(0.1560) Accuracy :  tensor(0.9467)
1560 Loss :  tensor(0.1559) Accuracy :  tensor(0.9467)
1570 Loss :  tensor(0.1557) Accuracy :  tensor(0.9467)
1580 Loss :  tensor(0.1556) Accuracy :  tensor(0.9467)
1590 Loss :  tensor(0.1554) Accuracy :  tensor(0.9467)
1600 Loss :  tensor(0.1552) Accuracy :  tensor(0.9467)
1610 Loss :  tensor(0.1551) Accuracy :  tensor(0.9467)
1620 Loss :  tensor(0.1549) Accuracy :  tensor(0.9467)
1630 Loss :  tensor(0.1548) Accuracy :  tensor(0.9467)
1640 Loss :  tensor(0.1546) Accuracy :  tensor(0.9467)
1650 Loss :  tensor(0.1545) Accuracy :  tensor(0.9467)
1660 Loss :  tensor(0.1543) Accuracy :  tensor(0.9465)
1670 Loss :  tensor(0.1542) Accuracy :  tensor(0.9465)
1680 Loss :  tensor(0.1540) Accuracy :  tensor(0.9465)
1690 Loss :  tensor(0.1539) Accuracy :  tensor(0.9465)
1700 Loss 

3130 Loss :  tensor(0.1397) Accuracy :  tensor(0.9493)
3140 Loss :  tensor(0.1396) Accuracy :  tensor(0.9496)
3150 Loss :  tensor(0.1396) Accuracy :  tensor(0.9493)
3160 Loss :  tensor(0.1395) Accuracy :  tensor(0.9493)
3170 Loss :  tensor(0.1395) Accuracy :  tensor(0.9493)
3180 Loss :  tensor(0.1394) Accuracy :  tensor(0.9493)
3190 Loss :  tensor(0.1394) Accuracy :  tensor(0.9493)
3200 Loss :  tensor(0.1393) Accuracy :  tensor(0.9498)
3210 Loss :  tensor(0.1392) Accuracy :  tensor(0.9498)
3220 Loss :  tensor(0.1392) Accuracy :  tensor(0.9498)
3230 Loss :  tensor(0.1391) Accuracy :  tensor(0.9498)
3240 Loss :  tensor(0.1391) Accuracy :  tensor(0.9498)
3250 Loss :  tensor(0.1390) Accuracy :  tensor(0.9498)
3260 Loss :  tensor(0.1390) Accuracy :  tensor(0.9501)
3270 Loss :  tensor(0.1389) Accuracy :  tensor(0.9501)
3280 Loss :  tensor(0.1389) Accuracy :  tensor(0.9498)
3290 Loss :  tensor(0.1388) Accuracy :  tensor(0.9496)
3300 Loss :  tensor(0.1388) Accuracy :  tensor(0.9498)
3310 Loss 

4650 Loss :  tensor(0.1356) Accuracy :  tensor(0.9506)
4660 Loss :  tensor(0.1356) Accuracy :  tensor(0.9506)
4670 Loss :  tensor(0.1356) Accuracy :  tensor(0.9506)
4680 Loss :  tensor(0.1356) Accuracy :  tensor(0.9506)
4690 Loss :  tensor(0.1356) Accuracy :  tensor(0.9506)
4700 Loss :  tensor(0.1356) Accuracy :  tensor(0.9506)
4710 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4720 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4730 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4740 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4750 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4760 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4770 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4780 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4790 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4800 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4810 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4820 Loss :  tensor(0.1355) Accuracy :  tensor(0.9506)
4830 Loss 

In [11]:
Predict = model(TEST)
print(Predict)

tensor([[2.4370e-06],
        [4.4240e-02],
        [1.0229e-01],
        ...,
        [9.6703e-01],
        [1.5967e-05],
        [4.2057e-05]], grad_fn=<SigmoidBackward0>)


In [12]:
predicts = pd.DataFrame(Predict.data.numpy(),columns=["type"])
print(predicts)
temp_white = predicts["type"] <= 0.5
temp_red = predicts["type"] > 0.5
predicts[temp_white]="white"
predicts[temp_red]="red"
print(predicts)
predicts.to_csv("G:/내 드라이브/연구실/PyTorch/2주차/Test_Predict.csv",sep=",",index=False)

          type
0     0.000002
1     0.044240
2     0.102288
3     0.000068
4     0.001526
...        ...
1645  0.000105
1646  0.005976
1647  0.967033
1648  0.000016
1649  0.000042

[1650 rows x 1 columns]
       type
0     white
1     white
2     white
3     white
4     white
...     ...
1645  white
1646  white
1647    red
1648  white
1649  white

[1650 rows x 1 columns]
